In [0]:
%pip install openai unidecode gspread==5.12.4 tiktoken mlflow
%restart_python
%load_ext autoreload
%autoreload 2 

In [0]:
%restart_python
%load_ext autoreload
%autoreload 2 

In [0]:
from abc import ABC, abstractmethod
import time
import mlflow
from contextlib import contextmanager
import json
import pandas as pd
import datetime
import re
import openai
from openai import OpenAI
import gspread
import random
import logging
import os
from unidecode import unidecode
from pathlib import Path
import pyspark
from pyspark.sql.functions import *
from functools import reduce
from typing import *
import tiktoken


from general_config import *
from ml_tracker import MLTracker
from aso_config import *
from aso_localizer import *
#from in_game_localizer import InGameLocalizer
#from InGame_Config import * 

In [0]:
%run "./authenticationScript"

In [0]:
#TODO update authenticatinon script and secrets!!
gsheet_client = get_gspread_client_from_secret_old()
gpt_client = get_model_client() 

In [0]:
request =  {"RowFingerprint":"testing_aso_v1",
                    "Timestamp":datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    "SubmitterEmail":"krista@jamcity.com",
                    "DueDate":"8/29/25",
                    'LocType':"ASO", 
                    'Game':"Panda Pop", 
                    'TargetLanguages':"",
                    "URL":"https://docs.google.com/spreadsheets/d/1QsZqlJ_vZC5fYIKnSFdhkUxi-UAIRNzTB24DDyoZMhU/edit?gid=980819314#gid=980819314",
                    "QAFlag":""}


cfg = {
    "input": 
        {
            "required_tabs": ["ios","android"],
            "ios_header_rows": 3, 
            "android_header_rows": 3
        },
    ##add more formatting data for header rows
    "char_limit_policy": "strict",
    "output_sheets":
        ["formatted ios", "formatted android", "long results",'wide results'],
    ##add more formatting info for output sheets
    }

In [0]:
localizer = ASOLocalizer(request = request, gsheet_client =gsheet_client, gpt_client = gpt_client, cfg=cfg)

In [0]:
localizer.validate_inputs() # GOOD
localizer.load_inputs() #Good
test = localizer.preprocess(data=None) #Good

In [0]:
prompts = localizer.build_prompts(test) #Good
raw_results = localizer.translate(prompts) #Good

In [0]:
processed = localizer.postprocess(raw_results)
print(localizer.write_outputs(processed))
#localizer.format_results(processed)


#unioned_long, unioned_wide = localizer.format_results(processed)

In [0]:
#

In [0]:
wide_results, long_results = localizer.format_results(processed)

In [0]:
dbutils.notebook.exit()

In [0]:
wide_order = ['row_id','en_char_limit','game','platform','type_desc','en_US',*localizer.lang_cds]

long_order = ["RowFingerprint","row_idx",'row_id','en_char_limit','game','platform','type_desc','en_US','language','language_cd','target_char_limit','translation']


ordered_wide = wide_results[wide_order]
ordered_long = long_results[long_order]


data_long_range = f"A2:L{len(ordered_long)+1}"
long_data = ordered_long.values.tolist()


data_wide_range = f"A2:P{len(ordered_wide)+1}"
wide_data = ordered_wide.values.tolist()

wide_wksht = localizer.sh.worksheet('wide results')
long_wksht = localizer.sh.worksheet('long results')


wide_wksht.batch_update([{'range':data_wide_range, 'values':wide_data}])
long_wksht.batch_update([{'range':data_long_range, 'values':long_data}])